# PaLM APIの紹介
本紹介は，ツールの歴史的背景や詳細な技術的・構成的情報は簡素化し，ツール利用者向けのチュートリアルを指向しています．ご了承ください．

## 最初に
本ツールの利用は有料になる可能性があります．今回はGoogle Vertex AIの初期無料クレジット300$を利用することを前提にして話を進めます．

## PaLM APIとは
ここでは，Google Cloud Vertex AIにて提供される大規模言語モデル (Large-Language Model; LLM)をPythonプログラムからアクセスするAPIを紹介し，特に``PaLM2 Text Bison``モデルの利用方法を紹介することから，これをPaLM APIと呼称して紹介します．

PaLMとは，Pathways Language Modelの略称でLLMの一つで，PaLM2はPaLMの後継モデルとなります．（前身の）PaLMの時点で，5,400億個のパラメタを採用し，7,800億個のトークンを学習データに用いており，その規模の大きさがわかります．後継のPaLM2では，100種以上の多言語に対応し，科学論文や数式も学習しているために論理・推論・数学的能力が向上しているとも言われています．

PaLM APIの利用は，Googleが提供するWebサービスを通じて利用するため，Google Colabに閉じた手順にはなりません．また，商用化されている製品ですので，当該サービスへ入出力されるトークン量に応じて課金されます（従量課金制）．公式サイトによると，``入力 1,000 文字（プロンプト）と出力 1,000 文字（レスポンス）ごとに課金されます。文字数は UTF-8 コードポイントでカウントされ、空白文字はカウントから除外されます。``とあり，``PaLM 2 Text Bison``では，入出力のどちらも1,000文字あたり$0.0010（≒0.14円）（執筆時点）かかります．単純計算で10万文字（入出力合わせて）で約14円ですから，リーズナブルなのかもしれません．このような製品ツールであることを留意してください．

## 利用手順

1. https://www.google.com/intl/ja/account/about/ から個人用のGoogleアカウントを一つ用意してください．（既にGoogleアカウントを所有しており，それを利用する場合は本ステップは飛ばして構いません．ただし，組織が提供するアカウントを利用する場合は組織ポリシーに応じて正常に進められないステップがあり得ます．）
1. https://cloud.google.com/vertex-ai?hl=ja にて，ステップ1で作成したアカウントにログインの上，``Vertex AI 無料トライアル``を選択して，Vertex AIの利用を開始します．利用を開始するまでの手続きに，クレジットカード情報や住所を入力することがありますので留意してください．クレジットカードが利用されるケースについては，その入力を求められる画面で確認してください．
1. Vertex AIのトップページが表示されたら，左側メニューの``TOOLS`` > ``Model Garden``を選択し，``基盤モデル``中の``すべて表示(数字)``を選択します．展開された表示の中で，``PaLM2 Text Bison``の項目を探し，``詳細を表示``を選択してください．
1. ここでは``Documentation`` > ``Setup the PaLM 2 Text Bison`` > ``Setup the PaLM 2 Text Bison (Python)`` > ``Option 2: Vertex AI Python SDK with Google Colab``の内容を参考に話を進めます．他の環境でセットアップは，他の項を参照してみてください．
1. 前ステップの情報からわかるように，``Vertex AI``を通じて``PaLM2 Text Bison``（以下，PaLM APIと呼称）をPythonプログラムから利用するには，Project IDが必要になります．Project IDの確認方法の一つは，``Vertex AI``のトップページ上でページ最上部に``Google Cloud``ロゴ右側に（おそらく）``My First Project``という選択肢が既定になっているドロップダウンメニューのような項目があります．それ（選択済のプロジェクト名）を選択すると，ダイアログが表示され，IDが見えるようになります．（おそらく，書式は<英単語>-<英単語>-<数値>のはず．）これを控えておいてください．
1. Vertex AI APIは最初無効化されていますので，有効化する必要があります．``Vertex AI``のトップページ上でページ最上部中央に検索フォームに``Vertex AI API``と入力して，検索してください．検索結果で``Vertex AI API``が表示されたら選択し，``有効にする``を選択してください．以上で，本資料に掲載するコードを実行する準備ができました．

In [ ]:
# Vertex AIをPythonプログラムから利用するために，Vertex AI SDK for Pythonをインストールします．
!pip install google-cloud-aiplatform

In [ ]:
# 最初にGoogle認証を行う．Vertex AIのサービスは個別のGoogleアカウントに紐づいているため，利用には認証が必要．
from google.colab import auth as google_auth
google_auth.authenticate_user()

# Vertex AI SDK for Pythonのライブラリをimportする
# APIドキュメント: https://cloud.google.com/python/docs/reference/aiplatform/latest
# TextGenerationModel: https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.TextGenerationModel
import vertexai
from vertexai.language_models import TextGenerationModel

def predict_large_language_model_sample(
    project_id: str, # 自身のProject IDを入力．上記，利用手順5.を参照．
    model_name: str, # PaLM2 text bisonを利用する場合は"text-bison@002"とする．他のモデルの利用は利用手順3.のModel Gardenを参照．
    temperature: float, # 1に近いほどランダム性が高い回答が得られ，0に近いほどランダム性が低い回答が得られる．通常は0.2が推奨される．
    max_output_tokens: int, # 1回の出力における最大のトークン量を指定する．PaLM2 text bisonでの最大値は2048．
    top_p: float, # 確率上位のトークン累積確率の閾値．この閾値を超える最小のトークン数を選択候補にする．
    top_k: int, # 選択候補とする上位トークンの個数
    content: str, # 質問内容
    location: str = "asia-northeast1", # 東京にあるエンドポイント
    ) :

    vertexai.init(project=project_id, location=location) # 利用するプロジェクトIDとエンドポイントで指定して初期化
    model = TextGenerationModel.from_pretrained(model_name) # モデル名を基に訓練済モデルを取得（今回はPaLM 2 texgt bison）
    response = model.predict( # 質問に対する回答を取得
        content,
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        top_k=top_k,
        top_p=top_p)
    print(f"解答:\n{response.text}")
predict_large_language_model_sample("avid-pager-408809", "text-bison@002", 0.2, 2048, 0.8, 40, '''日本の都道府県と県庁所在地をテーブル形式で''')
# 注：出力が正しいとは限りません．

## 演習
トークンの消費量に注意してください．
1. モデル等のパラメタを様々に変えて，生成AIからの応答内容の質を比較しなさい．

### 参考文献
1. Aakanksha Chowdhery et al., PaLM: Scaling Language Modeling with Pathways, https://arxiv.org/abs/2204.02311
1. Google, Introducing PaLM 2, https://blog.google/technology/ai/google-palm-2-ai-large-language-model/
1. Google, Vertex AI, https://cloud.google.com/vertex-ai?hl=ja
1. Google, Vertex AIの料金, https://cloud.google.com/vertex-ai/pricing?hl=ja
1. Google, google-cloud-aiplatform, https://cloud.google.com/python/docs/reference/aiplatform/latest